In [375]:
import json
from functools import partial
from inspect import getsource
from math import log
from sys import argv
from bisect import bisect_left
import re


def checker(nm,ori):
    flag = False
    for key in ori.keys():
        if ori[key] != nm[key]:
            flag = True
            break
    print('Match!!' if flag == False else 'Error!!')

def consecu_int_run_length(idx):
    mark = []
    flag = False
    temp = [ 0, 0, 0 ]
    for i in range(len(k)):
        if i != idx[i]:
            if flag == False:
                temp = [ i, idx[i], 1 ]
                flag = True
            elif flag == True and idx[i]-(temp[1]+temp[2]) == 0:
                temp[2] += 1
            elif flag == True:
                mark += temp
                
                #print(temp)
                temp = [ i, idx[i], 1 ]
                flag = True
        else:
            if flag == True:
                mark += temp
                #print(temp)
                flag = False
    return mark

def sporadic_seg_run_length(mark):
    sporadic_seg = [ mark[i:i+3] for i in range(0,len(mark),3)]
    flag = False
    temp = []
    res = []
    for i in range(len(sporadic_seg)):
        if sporadic_seg[i][2] != 1:
            if flag == True:
                res += temp 
                # initialize temp to an empty list, to avoid if temp have something but index is at the end
                temp = []
                flag = False
            res += sporadic_seg[i]
        else:
            # first one
            if flag == False:
                # start index, index list, len
                temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
                flag = True
            else:
                if sporadic_seg[i][0] - sporadic_seg[i-1][0] == 1:
                    temp[1].append(sporadic_seg[i][1]) 
                else:
                    res += temp  
                    temp = [sporadic_seg[i][0],[sporadic_seg[i][1]],1]
    if len(temp) != 0:
        res += temp  
    return res
        
if __name__ == '__main__':
    map_in_name, script_name = '../cases/case1.json', 'test.py'
    nf = '../input/case1_in.json'
    nf = open(nf, 'r')
    nf = json.load(nf)
    keys, values = sorted(nf[0]), sorted(nf[1])
    
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()

    mmm = sorted(list(origin_mapping.items()),key=lambda x: x[0])

    k,v = zip(*mmm)
    mm = sorted(zip(v,[ i for i in range(len(k)) ]),key=lambda x: x[0])
    v, idx = zip(*mm)
    idx = list(idx)
    
    mark = consecu_int_run_length(idx)
    # check part
    _m = mark
    nm = {}
    for i in range(len(keys)):
        nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        for j in range(_m[i+2]):
            nm[keys[_m[i+1]+j]] = values[_m[i]+j]
    check(nm,origin_mapping)

    sporadic_seg = sporadic_seg_run_length(mark)
    # check part
    _m = sporadic_seg
    nm = {}
    for i in range(len(keys)):
        nm[keys[i]] = values[i]
    for i in range(0,len(_m),3):
        if _m[i+2] != 1:
            for j in range(_m[i+2]):
                nm[keys[_m[i+1]+j]] = values[_m[i]+j]
        else:
            length = len(_m[i+1])
            for j in range(length):
                nm[keys[_m[i+1][j]]] = values[_m[i]+j]
    check(nm,origin_mapping)
    document = """
import json
import hashlib
from sys import argv

_m = {}
nf, of = argv[1], argv[2]
nf = open(nf, 'r')
nf = json.load(nf)
keys, values = sorted(nf[0]), sorted(nf[1])

nm = {{}}
for i in range(len(keys)):
    nm[keys[i]] = values[i]
for i in range(0,len(_m),3):
    if _m[i+2] != 1:
        for j in range(_m[i+2]):
            nm[keys[_m[i+1]+j]] = values[_m[i]+j]
    else:
        length = len(_m[i+1])
        for j in range(length):
            nm[keys[_m[i+1][j]]] = values[_m[i]+j]
of = open(of, 'w')
of.write(json.dumps(nm))
of.close()
    """.format(str(sporadic_seg))
    script = open(script_name, 'w')
    script.write(document)
    script.close()

Match!!
50161 46243
362
Match!!
